# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Cleaning and manipulating data before exercise

First inspecting the data.

In [2]:
df = pd.read_csv('jeopardy.csv')
#Inspect the data
df.head()

#Column heads are capitalised and have spaces

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,"No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves",Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,"The city of Yuma in this state has a record average of 4,055 hours of sunshine each year",Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", this company served its billionth burger",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Constitution of Mass., second President of the United States",John Adams


In [3]:
#Count the data
print(len(df))

#Data has 216930 records

#Investigate data types
df.dtypes

216930


Show Number     int64
 Air Date      object
 Round         object
 Category      object
 Value         object
 Question      object
 Answer        object
dtype: object

Data has 216930 records. The column names are capitalised and have blank spaces. Data types are also all objects. 

**First data prep step: rename column names**

In [4]:
#First strip out leading or trailing white space from column names
df.columns = df.columns.str.strip()

#Then rename columns
df.rename(columns={
    'Show Number': 'show_number',
    'Air Date' : 'air_date',
    'Round': 'round',
    'Category': 'category',
    'Value': 'value',
    'Question': 'question',
    'Answer': 'answer'
}, inplace=True)
df.head()

,show_number,air_date,round,category,value,question,answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,"No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves",Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,"The city of Yuma in this state has a record average of 4,055 hours of sunshine each year",Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", this company served its billionth burger",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Constitution of Mass., second President of the United States",John Adams


**Second data prep step: change data types**

In [5]:
#Create dictionary of new datatypes
dtype_changes = {
    'show_number': 'int',
    'round': 'str',
    'category': 'str',
    'value': 'float',
    'question': 'str',
    'answer':'str'
}

#Strip $ from value column to allow conversion to int
df['value'] = df['value'].str.replace('$', '').str.replace(',','')

#Convert non-numeric values in value coumn to NaN
df['value'] = pd.to_numeric(df['value'], errors='coerce')

#Convert date column to date type
df['air_date'] = pd.to_datetime(df['air_date'])

#Convert other columns to data types declared before
df = df.astype(dtype_changes)

#Inspect new datatypes
df.dtypes

show_number             int64
air_date       datetime64[ns]
round                  object
category               object
value                 float64
question               object
answer                 object
dtype: object

### Analysis and Exercise

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [6]:
def filter_questions_by_string(df, column_name, strings):
    #Boolean mask for rows containing all strings
    mask = df[column_name].apply(lambda x: all(s in x for s in strings))
    
    #filter dataframe using mask
    return df[mask]

#Test code
questions_test = filter_questions_by_string(df, 'question', ['100'])
print(f'There are {len(questions_test)} records.')
questions_test['question'].head()

There are 911 records.


12            In 1000 Rajaraja I of the Cholas battled to take this Indian Ocean island now known for its tea
34      Around 100 A.D. Tacitus wrote a book on how this art of persuasive speaking had declined since Cicero
1011                                       Strike up the band; this Christian organization turned 100 in June
2412                                 When it opened, it cut the distance from London to Bombay by 5,100 miles
2448    This city, first mentioned in 912, is the seat of Britain's oldest university, developed in the 1100s
Name: question, dtype: object

3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

**Make search function case insensitive.**

In [7]:
def filter_questions_case_insensitive(df, column_name, strings):
    #convert strings to lowercase
    strings = [s.lower() for s in strings]
    
    #Boolean mask for rows containing strings (case-insensitive)
    mask = df[column_name].apply(lambda x: all(s in x.lower() for s in strings))
    
    #Return filtered dataframe
    return df[mask]

#Test code
case_insensitive_test = filter_questions_case_insensitive(df, 'question', ['EnGlAnd', 'kInG'])
print(f'There are {len(case_insensitive_test)} records.')
case_insensitive_test.head()

There are 152 records.


,show_number,air_date,round,category,value,question,answer
4953,3003,1997-09-24,Double Jeopardy!,"""PH""UN WORDS",200.0,"Both England's King George V & FDR put their stamp of approval on this ""King of Hobbies""",Philately (stamp collecting)
6337,3517,1999-12-14,Double Jeopardy!,Y1K,800.0,"In retaliation for Viking raids, this ""Unready"" king of England attacks Norse areas of the Isle of Man",Ethelred
9191,3907,2001-09-04,Double Jeopardy!,WON THE BATTLE,800.0,This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt,Henry V
11710,2903,1997-03-26,Double Jeopardy!,BRITISH MONARCHS,600.0,"This Scotsman, the first Stuart king of England, was called ""The Wisest Fool in Christendom""",James I
13454,4726,2005-03-07,Jeopardy!,A NUMBER FROM 1 TO 10,1000.0,It's the number that followed the last king of England named William,4


**Make search function not search substrings**

In [8]:
def filter_questions_no_substrings(df, column_name, strings):
    #convert search strings to lower case
    strings = [s.lower() for s in strings]
    
    #Boolean mask for rows containing strings or their possessive forms and splits words
    def check_words(text):
        words = text.lower().split()
        return all(any(s == word or s + "''" == word for word in words) for s in strings)
    
    mask = df[column_name].apply(check_words)
    
    
    #Return filtered dataframe
    return df[mask]

#Test code
case_substring_test = filter_questions_no_substrings(df, 'question', ['king', 'england'])
print(f'There are {len(case_substring_test)} records.')
case_substring_test.head()

There are 74 records.


,show_number,air_date,round,category,value,question,answer
6337,3517,1999-12-14,Double Jeopardy!,Y1K,800.0,"In retaliation for Viking raids, this ""Unready"" king of England attacks Norse areas of the Isle of Man",Ethelred
9191,3907,2001-09-04,Double Jeopardy!,WON THE BATTLE,800.0,This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt,Henry V
13454,4726,2005-03-07,Jeopardy!,A NUMBER FROM 1 TO 10,1000.0,It's the number that followed the last king of England named William,4
14912,2832,1996-12-17,Jeopardy!,WORLD HISTORY,100.0,"This country's King Louis IV was nicknamed ""Louis From Overseas"" because he was raised in England",France
18076,3227,1998-09-22,Double Jeopardy!,WORLD HISTORY,1000.0,In 1199 this crusader king of England was mortally wounded while besieging the castle of Chalus,Richard the Lionhearted


4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   While most of the values in the `" Value"` column represent a dollar amount as a string, note that some do not &mdash; these values will need to be handled differently!

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.